> Este ntb gera ``model_annotations_merged.csv`` , unindo `PATH_SUMMEVAL_JSONL` com `*.storys` 

Dados de storys podem ser baixados em https://github.com/ThomasScialom/QuestEval 

In [ ]:
import pandas as pd
from research.src.constants import PATH_SUMMEVAL_JSONL
df = pd.read_json(PATH_SUMMEVAL_JSONL, lines=True)


c:\Users\thiago.ouverney\Projetos\pyAutoSummarizer\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The following layers were not sharded: encoder.layer.*.attention.attn.v.weight, encoder.layer.*.output.dense.weight, pooler.dense.weight, encoder.layer.*.intermediate.dense.weight, embeddings.LayerNorm.weight, pooler.dense.bias, encoder.relative_attention_bias.weight, embeddings.LayerNorm.bias, encoder.layer.*.attention.attn.q.weight, encoder.layer.*.attention.attn.v.bias, embeddings.position_embeddings.weight, encoder.layer.*.attention.attn.q.bias, encoder.layer.*.attention.attn.k.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.attn.o.bias, encoder.layer.*.attention.LayerNorm.bias, encoder.layer.*.attention.LayerNorm.weight, encoder.layer.*.output.LayerNorm.weight, embeddings.word_embe

In [ ]:
df[df.id == "dm-test-00dad5d893f836f53080de50e9b983ad2f9bcd22"]

,id,decoded,expert_annotations,turker_annotations,references,model_id,filepath


In [ ]:
df.tail()

,id,decoded,expert_annotations,turker_annotations,references,model_id,filepath
1595,dm-test-e880fda4c25289f8325574246f0f8ed4ff5eb26b,a timewarp home which has remained unchanged s...,"[{'coherence': 5, 'consistency': 5, 'fluency':...","[{'coherence': 4, 'consistency': 3, 'fluency':...",[Nondescript semi-detached home for sale in Ho...,M0,cnndm/dailymail/stories/e880fda4c25289f8325574...
1596,dm-test-e880fda4c25289f8325574246f0f8ed4ff5eb26b,"The collector 's paradise in Horfield , Bristo...","[{'coherence': 3, 'consistency': 5, 'fluency':...","[{'coherence': 4, 'consistency': 4, 'fluency':...",[Nondescript semi-detached home for sale in Ho...,M22,cnndm/dailymail/stories/e880fda4c25289f8325574...
1597,dm-test-e880fda4c25289f8325574246f0f8ed4ff5eb26b,"the collector’s paradise in horfield , bristol...","[{'coherence': 5, 'consistency': 5, 'fluency':...","[{'coherence': 4, 'consistency': 4, 'fluency':...",[Nondescript semi-detached home for sale in Ho...,M8,cnndm/dailymail/stories/e880fda4c25289f8325574...
1598,dm-test-e880fda4c25289f8325574246f0f8ed4ff5eb26b,"the collector 's paradise in horfield , bristo...","[{'coherence': 3, 'consistency': 5, 'fluency':...","[{'coherence': 4, 'consistency': 3, 'fluency':...",[Nondescript semi-detached home for sale in Ho...,M10,cnndm/dailymail/stories/e880fda4c25289f8325574...
1599,dm-test-e880fda4c25289f8325574246f0f8ed4ff5eb26b,the timewarp home has remained unchanged since...,"[{'coherence': 3, 'consistency': 5, 'fluency':...","[{'coherence': 4, 'consistency': 3, 'fluency':...",[Nondescript semi-detached home for sale in Ho...,M9,cnndm/dailymail/stories/e880fda4c25289f8325574...


In [ ]:
df['folder_type'] = df.filepath.apply(lambda x: x.split("/")[1])

In [ ]:
df['folder_type'].value_counts()

folder_type
dailymail    1392
cnn           208
Name: count, dtype: int64

In [ ]:
import os
cndm_path = "" #"C:\\Users\\thiago.ouverney\\Desktop\\TCC\\cndm"
os.chdir(cndm_path)

In [ ]:
files = [f for f in os.listdir('cnn/stories') if f.endswith(".story")]
files_daily = [f for f in os.listdir('dailymail_stories/dailymail/stories') if f.endswith(".story")]
files += files_daily

C:/Users/thiago.ouverney/Desktop/TCC/cnn/stories
C:/Users/thiago.ouverney/Projetos/pyAutoSummarizer/data/model_annotations.aligned.jsonl

> Encontrando storys cm match

In [ ]:
len(files)

312085

In [ ]:
from tqdm import tqdm

lista_encontrados = []
for story_path in tqdm(df.filepath):
    for f in files:
        if story_path.split("/")[-1] == f:
            # print(f, story_path)
            lista_encontrados.append(story_path) 
            break

100%|██████████| 1600/1600 [00:51<00:00, 31.33it/s]


In [ ]:
print(f"Percentual de cobertura {100*len(lista_encontrados) / len(df.filepath) }%")

Percentual de cobertura 100.0%


In [ ]:
from pathlib import Path
import pandas as pd
import os
from tqdm import tqdm
# Diretórios onde estão os .story
ROOTS = [
    Path("cnn/stories"),
    Path("dailymail_stories/dailymail/stories"),
]

def build_story_df(roots):
    """
    Lê todos os .story, gera um DataFrame com story_id, content e filepath.
    """
    rows = []
    for root in roots:
        for p in tqdm(root.rglob("*.story")):          # rglob cobre sub-pastas, se existirem
            story_id = p.stem                    # ex.: 'abc123' de 'abc123.story'
            with p.open(encoding="utf-8") as fh:
                content = fh.read()
            rows.append(
                {"story_id": story_id,
                 "content": content,
                 "filepath_story": str(p)}
            )
    return pd.DataFrame(rows)

stories_df = build_story_df(ROOTS)
print(stories_df.head())


92579it [01:28, 1049.23it/s]
219506it [07:29, 488.11it/s] 


                                   story_id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   
1  0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2  00027e965c8264c35cc1bc55556db388da82b07f   
3  0002c17436637c4fe1837c935c04de47adb18e9a   
4  0003ad6ef0c37534f80b55b4235108024b407f0b   

                                             content  \
0  It's official: U.S. President Barack Obama wan...   
1  (CNN) -- Usain Bolt rounded off the world cham...   
2  Kansas City, Missouri (CNN) -- The General Ser...   
3  Los Angeles (CNN) -- A medical doctor in Vanco...   
4  (CNN) -- Police arrested another teen Thursday...   

                                      filepath_story  
0  cnn\stories\0001d1afc246a7964130f43ae940af6bc6...  
1  cnn\stories\0002095e55fcbd3a2f366d9bf92a95433d...  
2  cnn\stories\00027e965c8264c35cc1bc55556db388da...  
3  cnn\stories\0002c17436637c4fe1837c935c04de47ad...  
4  cnn\stories\0003ad6ef0c37534f80b55b4235108024b...  


Usando o exemplo 

`'cnndm/dailymail/stories/fbbafa743a8c2ecd2cedf65c6c61956b2db8ec5c.story'`


In [ ]:
df["story_id"] = (df["filepath"]
                  .str.split("/")            # quebra por barras
                  .str[-1]                   # pega o último item
                  .str.replace(".story", "", regex=False))

# Lista de story_ids que realmente existem nos diretórios
ids_validos = set(stories_df["story_id"])

# Equivalente ao seu lista_encontrados, mas vetorizado
df["existe_story"] = df["story_id"].isin(ids_validos)

In [ ]:
df_merged = (
    df.merge(stories_df[["story_id", "content"]],  # só traz o texto
             on="story_id",
             how="left",            # left join mantém todo o df original
             suffixes=("", "_story"))
)

In [ ]:
df_merged.existe_story.value_counts()

existe_story
True    1600
Name: count, dtype: int64

In [ ]:
df_merged.to_csv("C:\\Users\\thiago.ouverney\\Projetos\\pyAutoSummarizer\\data\\model_annotations_merged.csv")

In [ ]:
print(df_merged.loc[0,'content'])

Paul Merson has restarted his row with Andros Townsend after the Tottenham midfielder was brought on with only seven minutes remaining in his team's 0-0 draw with Burnley on Sunday.

'Just been watching the game, did you miss the coach? #RubberDub #7minutes,' Merson put on Twitter.

Merson initially angered Townsend for writing in his Sky Sports column that 'if Andros Townsend can get in (the England team) then it opens it up to anybody.'

Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley

Townsend was brought on in the 83rd minute for Tottenham as they drew 0-0 against Burnley

Andros Townsend scores England's equaliser in their 1-1 friendly draw with Italy in Turin on Tuesday night

The former Arsenal man was proven wrong when Townsend hit a stunning equaliser for England against Italy and he duly admitted his mistake.

'It's not as though I was watching hoping he wouldn't score for England, I'm genuinely pleased for him and fair play t

> Analisando referencia gerada por um modelo extrativo 

In [ ]:
decoded_m5 = df[(df.id=='cnn-test-fbbafa743a8c2ecd2cedf65c6c61956b2db8ec5c') & (df.model_id == 'M5')].decoded.values[0]
print(decoded_m5)

" roots , " the epic miniseries about an african - american slave and his descendants , had a staggering audience of over 100 million viewers back in 1977 . now a&e networks are remaking the miniseries , to air in 2016 .


In [ ]:
decoded_m20 = df[(df.id=='cnn-test-fbbafa743a8c2ecd2cedf65c6c61956b2db8ec5c') & (df.model_id == 'M20')].decoded.values[0]
print(decoded_m20)

A&E Networks will simulcast the original "Roots" in 2016. The original "Roots" premiered in 1977 and ran for four seasons. The miniseries followed Kunta Kinte, a free black man in Virginia, as he was sold into slavery.
